In [ ]:
#!pip3 install autokeras
#!pip3 install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import logging
logging.getLogger('tensorflow').disabled = True

from sklearn.datasets import load_files
from tensorflow.keras.datasets import mnist
import autokeras as ak

import plotly.express as px
import copy 
import os
import json

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc, precision_recall_curve, roc_auc_score, average_precision_score
from sklearn.datasets import make_classification
from plotly.subplots import make_subplots

def plot_roc(y, y_score):
    fpr, tpr, thresholds = roc_curve(y, y_score)

    fig = px.area(
        x=fpr, y=tpr,
        title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='False Positive Rate', y='True Positive Rate'),
        width=700, height=500
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    fig.show()
    
def plot_pr_curve(y, y_score):    
    precision, recall, thresholds = precision_recall_curve(y, y_score)

    fig = px.area(
        x=recall, y=precision,
        title=f'Precision-Recall Curve (AUC={auc(recall, precision):.4f})',
        labels=dict(x='Recall', y='Precision'),
        width=700, height=500
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=1, y1=0
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    fig.show()    
    
def plot_roc_train_test(y_train, y_score_train, y_test, y_score_test):
    fig = go.Figure()
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )
    Y_Scores = [y_score_train, y_score_test]
    Y_Obs = [y_train, y_test]
    Y_Names = ["Train", "Test"]
    for i in range(len(Y_Scores)):
        y_true = Y_Obs[i]
        y_score = Y_Scores[i]

        fpr, tpr, _ = roc_curve(y_true, y_score)
        auc_score = roc_auc_score(y_true, y_score)

        name = f"{Y_Names[i]} ROC (AUC={auc_score:.4f})"
        fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'))

    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=700, height=500
    )
    fig.show()
    
def plot_pr_train_test(y_train, y_score_train, y_test, y_score_test):
    #fig = go.Figure()
    fig = make_subplots(rows=1, cols=2)
    
    # subplot for ROC curve
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1,
        row=1, col=1
    )
    Y_Scores = [y_score_train, y_score_test]
    Y_Obs = [y_train, y_test]
    Y_Names = ["Train", "Test"]
    for i in range(len(Y_Scores)):
        y_true = Y_Obs[i]
        y_score = Y_Scores[i]

        fpr, tpr, _ = roc_curve(y_true, y_score)
        auc_score = roc_auc_score(y_true, y_score)

        name = f"{Y_Names[i]} ROC (AUC={auc_score:.4f})"
        fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'),row=1, col=1)
    
    fig.update_xaxes(title_text="False Positive Rate", row=1, col=1)
    fig.update_yaxes(title_text="True Positive Rate", row=1, col=1)
    
    # subplot for PR curve
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=1, y1=0, 
        row=1,col=2
    )
    Y_Scores = [y_score_train, y_score_test]
    Y_Obs = [y_train, y_test]
    Y_Names = ["Train", "Test"]
    for i in range(len(Y_Scores)):
        y_true = Y_Obs[i]
        y_score = Y_Scores[i]

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        auc_score = average_precision_score(y_true, y_score)
    
        name = f"{Y_Names[i]} PR (AUC={auc_score:.4f})"
        fig.add_trace(go.Scatter(x=recall, y=precision, name=name, mode='lines'),row=1,col=2)

    fig.update_xaxes(title_text="Recall", row=1, col=2)
    fig.update_yaxes(title_text="Precision", row=1, col=2)
    fig.update_layout(
        title="ROC & Precision-Recall Curves",
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=1000, height=500
    )
    fig.show()
    
def hyper_table(path='structured_data_classifier'):
    trial_json = os.popen('ls ./{}/trial_*/trial.json'.format(path)).read().split('\n')[:-1]
    DATA = []
    for file in trial_json:
        with open(file) as f: 
            DATA.append(json.load(f))
    for k in range(len(DATA)):
        DATA[k]['hyperparameters']['values']['score'] = DATA[k]['score']
    hyper_df = pd.concat([pd.DataFrame.from_dict(data['hyperparameters']['values'], orient='index') for data in DATA], axis=1)
    hyper_df.columns = ["trial#{}".format(k+1) for k in range(len(DATA))]
    return(hyper_df)

def predic_error_analysis(x_train, y_train, y_score_train, x_test, y_test, y_score):
    df1 = copy.deepcopy(x_train)
    df1['obs'] = y_train
    df1['predict'] = y_score_train
    df1['data'] = 'Train'
    #display(df1)
    df2 = copy.deepcopy(x_test)
    df2['obs'] = y_test
    df2['predict'] = y_score
    df2['data'] = 'Test'
    #display(df2)
    df3 = pd.concat([df1,df2])

    fig = px.scatter(
        df3, x='obs', y='predict',
        marginal_x='histogram', marginal_y='histogram',
        color='data', trendline='ols'
    )
    fig.update_traces(histnorm='probability', selector={'type':'histogram'})
    fig.add_shape(
        type="line", line=dict(dash='dash'),
        x0=df3['obs'].min(), y0=df3['obs'].min(),
        x1=df3['obs'].max(), y1=df3['obs'].max()
    )
    fig.update_layout(title="Prediction Error Analysis", 
                      yaxis=dict(range=[df3['obs'].min(), df3['obs'].max()]),
                      xaxis=dict(range=[df3['obs'].min(), df3['obs'].max()]),
                      width=1000, height=1000)
    fig.show()
    

# ROC PR curves for multi-class
def plot_pr_multi_class(y_train, y_score_train, y_test, y_score):

    y_class = np.unique(y_train)
    class_dict_test = {}
    for c in y_class:
        score = [s[c] for s in y_score_test]
        obs = y_test == c
        class_dict_test[c] = {'obs': obs, 'score': score}

    class_dict_train = {}
    for c in y_class:
        score = [s[c] for s in y_score_train]
        obs = y_train == c
        class_dict_train[c] = {'obs': obs, 'score': score}

    
    fig = make_subplots(rows=2, cols=2)
    
    # subplot for ROC curve: train data
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1,
        row=1, col=1)
    
    Y_Scores = [class_dict_train[c]['score'] for c in class_dict_train]
    Y_Obs = [class_dict_train[c]['obs'] for c in class_dict_train]
    Y_Names = y_class
    for i in range(len(Y_Scores)):
        y_true = Y_Obs[i]
        y_score = Y_Scores[i]

        fpr, tpr, _ = roc_curve(y_true, y_score)
        auc_score = roc_auc_score(y_true, y_score)

        name = f"class {Y_Names[i]} ROC: Train (AUC={auc_score:.4f})"
        fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'),row=1, col=1)
    
    fig.update_xaxes(title_text="False Positive Rate", row=1, col=1)
    fig.update_yaxes(title_text="True Positive Rate (Train)", row=1, col=1)

    # subplot for ROC curve: train data
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1,
        row=1, col=2)
    
    Y_Scores = [class_dict_test[c]['score'] for c in class_dict_test]
    Y_Obs = [class_dict_test[c]['obs'] for c in class_dict_test]
    Y_Names = y_class
    for i in range(len(Y_Scores)):
        y_true = Y_Obs[i]
        y_score = Y_Scores[i]

        fpr, tpr, _ = roc_curve(y_true, y_score)
        auc_score = roc_auc_score(y_true, y_score)

        name = f"class {Y_Names[i]} ROC: Test (AUC={auc_score:.4f})"
        fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'),row=1, col=2)
    
    fig.update_xaxes(title_text="False Positive Rate", row=1, col=2)
    fig.update_yaxes(title_text="True Positive Rate (Test)", row=1, col=2)

    # subplot for PR curve: train data
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=1, y1=0, 
        row=2,col=1)
    Y_Scores = [class_dict_train[c]['score'] for c in class_dict_train]
    Y_Obs = [class_dict_train[c]['obs'] for c in class_dict_train]
    Y_Names = y_class
    for i in range(len(Y_Scores)):
        y_true = Y_Obs[i]
        y_score = Y_Scores[i]

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        auc_score = average_precision_score(y_true, y_score)
    
        name = f"class {Y_Names[i]} PR Train (AUC={auc_score:.4f})"
        fig.add_trace(go.Scatter(x=recall, y=precision, name=name, mode='lines'),row=2,col=1)

    fig.update_xaxes(title_text="Recall", row=2, col=1)
    fig.update_yaxes(title_text="Precision (Train)", row=2, col=1)

    # subplot for PR curve: test data
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=1, y1=0, 
        row=2,col=2)
    Y_Scores = [class_dict_test[c]['score'] for c in class_dict_test]
    Y_Obs = [class_dict_test[c]['obs'] for c in class_dict_test]
    Y_Names = y_class
    for i in range(len(Y_Scores)):
        y_true = Y_Obs[i]
        y_score = Y_Scores[i]

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        auc_score = average_precision_score(y_true, y_score)
    
        name = f"class {Y_Names[i]} PR Test (AUC={auc_score:.4f})"
        fig.add_trace(go.Scatter(x=recall, y=precision, name=name, mode='lines'),row=2,col=2)

    fig.update_xaxes(title_text="Recall", row=2, col=2)
    fig.update_yaxes(title_text="Precision (Test)", row=2, col=2)

    fig.update_layout(
        title="ROC & Precision-Recall Curves",
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=1200, height=1000)
    
    fig.show()

## A Simple Example
The first step is to prepare your data. Here we use the MNIST dataset as an example


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

In [ ]:
import matplotlib.pyplot as plt
num = 20
k=np.random.randint(len(y_train))
images = x_train[k:(k+num)]
labels = y_train[k:(k+num)]
num_row = 2
num_col = 10
# plot images
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(images[i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[i]))
plt.tight_layout()
plt.show()

import plotly.express as px
df = pd.DataFrame(y_train, columns=['label'])
fig = px.histogram(df, x='label', nbins=10)
fig.show()

The second step is to run the ImageClassifier.
It is recommended have more trials for more complicated datasets.
This is just a quick demo of MNIST, so we set max_trials to 1.
For the same reason, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)

# Feed the image classifier with training data. That's it!
clf.fit(x_train, y_train, epochs=1)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


In [ ]:
# performance of the model
display(hyper_table(path='image_classifier'))

model = clf.export_model()
model.summary()

y_score_test = model.predict(x_test)
y_score_train = model.predict(x_train)

plot_pr_multi_class(y_train, y_score_train, y_test, y_score_test)

## Validation Data
By default, AutoKeras use the last 20% of training data as validation data. As
shown in the example below, you can use validation_split to specify the
percentage.

In [ ]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)


You can also use your own validation set instead of splitting it from the
training data with validation_data.


In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)


## Customized Search Space
For advanced users, you may customize your search space by using __AutoModel__
instead of ImageClassifier. You can configure the ImageBlock for some
high-level configurations, e.g., block_type for the type of neural network to
search, normalize for whether to do data normalization, augment for whether to
do data augmentation. You can also do not specify these arguments, which would
leave the different choices to be tuned automatically. See the following
example for detail.


In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=1)

In [ ]:
# performance of the model
display(hyper_table(path='auto_model'))

model = clf.export_model()
model.summary()

y_score_test = model.predict(x_test)
y_score_train = model.predict(x_train)

plot_pr_multi_class(y_train, y_score_train, y_test, y_score_test)

The usage of AutoModel is similar to the functional API of Keras. Basically, you are
building a graph, whose edges are blocks and the nodes are intermediate outputs of
blocks. To add an edge from input_node to output_node with output_node =
ak.[some_block]([block_args])(input_node).

You can even also use more fine grained blocks to customize the search space even
further. See the following example.


In [ ]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=1)

# performance of the model
display(hyper_table(path='auto_model'))

model = clf.export_model()
model.summary()

y_score_test = model.predict(x_test)
y_score_train = model.predict(x_train)

plot_pr_multi_class(y_train, y_score_train, y_test, y_score_test)

## Data Format
The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel
dimension. The images in the MNIST dataset do not have the channel dimension.
Each image is a matrix with shape (28, 28). AutoKeras also accepts images of
three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28,
28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e.
strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and
1s.

So if you prepare your data in the following way, the ImageClassifier should
still work.


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

# One-hot encode the labels.
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])


We also support using tf.data.Dataset format for the training data.


In [ ]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,)))
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,)))

clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=1)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

In [ ]:
# performance of the model
display(hyper_table(path='image_classifier'))

model = clf.export_model()
model.summary()

y_score_test = model.predict(x_test)
y_score_train = model.predict(x_train)

plot_pr_multi_class([np.argmax(y) for y in y_train], y_score_train, [np.argmax(y) for y in y_test], y_score_test)

## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/block/#normalization-class),
[ImageAugmentation](/block/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/block/#classificationhead-class).
